In [1]:
%%capture
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [3]:
from huggingface_hub import login
hf_token=""
login(token = hf_token)

base_model="maywell/Llama-3-Ko-8B-Instruct"
dataset_name = "./custom_datasets"
new_model = "llama-3-8b-chat-doctor"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [5]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [6]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [7]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [8]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(1000)) # Only use 1000 samples for quick demo

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][3]

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

'<|im_start|>user\n저는 VA 의사로부터 척수 감염에 대한 진단을 받았습니다. 그들은 일관되게 바이옵시와 MRI를 했습니다. 그들은 그것이 황색포도상구균이나 암일 수 있다고 말합니다. 하지만 그들은 2013년 12월 이후로 그것이 무엇인지 정확히 알지 못하고 여전히 저에게 직접적인 답변을 줄 수 없습니다. 그들이 치료를 받는 동안 저에게 주는 약은 옥시코돈과 인터벤스 항생제입니다. 저는 그들이 저를 산부인과 의사로 사용하는 것이 매우 걱정됩니다. 저는 수수료 없이 조언을 받을 수 있을 것이라고 생각했습니다. 저는 지불할 신용 카드가 없습니다. 귀하의 문제에 대해 죄송합니다.<|im_end|>\n<|im_start|>assistant\n안녕하세요, 귀하의 상황을 이해할 수 있고 죄송합니다. 솔직히 말해서, 두 가지를 구별하기 어려운 경우가 많습니다. 가장 좋은 방법은 생검 자료를 배양을 위해 보내고 유기체의 성장을 살펴보는 것입니다. 황색포도상구균은 치료가 가능하므로 항생제 투여 후 반응이 없으면 암과 같은 다른 대안을 시사합니다. 도움이 되었기를 바랍니다.<|im_end|>\n'

In [9]:
dataset = dataset.train_test_split(test_size=0.1)

In [14]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

In [16]:
trainer.train()

Step,Training Loss,Validation Loss
90,1.795000,1.943703
180,1.933200,1.881462
270,1.867700,1.817647
360,1.716200,1.767248
450,2.045300,1.753404


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=450, training_loss=1.9038783672120836, metrics={'train_runtime': 474.6502, 'train_samples_per_second': 1.896, 'train_steps_per_second': 0.948, 'total_flos': 1.1909830520365056e+16, 'train_loss': 1.9038783672120836, 'epoch': 1.0})

In [17]:
model.config.use_cache = True

In [22]:
messages = [
    {
        "role": "user",
        "content": "안녕하세요. 선생님. 여드름이 심해서 왔어요. 어떻게 치료하죠?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False,
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True,
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=1000,
                         num_return_sequences=1, repetition_penalty=1.2)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


안녕, HCM에 오신 것을 환영합니다! 귀하는 여드름을 앓고 있습니다. 이 질환은 피부의 모낭에서 발생하며, 이는 땀샘이나 머리카락과 같은 다른 부위와는 다릅니다. 이러한 병변에는 붉어짐, 가려움증 및 따끔거림이 나타날 수 있으며, 때로는 감염으로 인해 염증이 생길 수도 있고, 그 결과 흰색 또는 노란색 점액질로 채워집니다. 일반적으로 1~2주 동안 지속됩니다. 증상 완화를 위해 항생제를 복용할 필요가 있을 것입니다. 또한 화장품이나 기저귀 등과 같이 피지 분비물이 많은 제품 사용을 피해야 합니다. 질문에 답변이 되었기를 바랍니다. 추가 설명이 있으면 언제든지 저에게 편지를 쓰세요. 감사합니다 - SRRaveendran 박사, 성학자/피부 과학자. 신경과 전문의. 호르몬 비만 전문의. 가정의학과 의사. 산부인과 의사. 소아과 의사. 외과의. 정형외과 의사. 물리치료사. 임상의. 건강 관리 전문가. 상담심리학자. 정신과 의사. 약국 종업원. 간호사. 조산사. 방사선 기술자. 응급구조 요원이면 누구라도 문의하세요. 최고의 경의를 표하고 싶습니다. 잘 지내시기 바라며, 좋은 하루 보내십시오. 감사합니다. Dr.S.R.Raveendran 박사, DCH., MSc(USA), MD(India). 성학자/성병 전문의. 호르몬 비만 전문의. 가정의학과 의사. 산부인과 의사. 소아과 의사. 외과의. 정형외과 의사. 물리치료사. 임상의. 건강 관리 전문가. 상담심리학자. 정신과 의사. 약국 종업원. 간호사. 조산사. 방사선 기술자. 응급구조 요원의 경우 누구라도 문의하세요. 최고의 경의를 표하고 싶습니다. 잘 지내시기 바라며, 좋은 하루 보내십시오. 감사합니다. Dr.S.R.Raveendran 박사, DCH., MSc(USA), MD(India) 성학자/성병 전문의. 호르몬 비만 전문의. 가정의학과 의사. 산부인과 의사. 소아과 의사. 외과의. 정형외과 의사. 물리치료사. 임상의. 건강 관리 전문가. 상담심리학자. 정신과 의사. 약국 종업원. 간호사. 조산사. 방사선 기술자.

In [23]:
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
